In [2]:
from PIL import Image
import os

def merge_images(year, period_combinations, output_patterns, input_base_path, output_base_path, spacing=10):
    """
    合并指定年份和时期组合的图片
    
    参数:
    year: 处理的年份
    period_combinations: 时期组合列表
    output_patterns: 输出文件名模式
    input_base_path: 输入图片的基础路径
    output_base_path: 输出图片的基础路径
    spacing: 图片之间的间距，默认为10像素
    """
    # 确保输出目录存在
    if not os.path.exists(output_base_path):
        os.makedirs(output_base_path)
    
    # 定义模型对比名称映射
    model_comparison_names = {
        'EQUATES_VNA_vs_EQUATES_eVNA_vs_EQUATES_aVNA_vs_EQUATES_Downscaler_vs_EQUATES_Harvard ML': 'EQUATES vs VNA eVNA aVNA Downscaler Harvard ML',
        'VNA_eVNA_vs_VNA_aVNA_vs_VNA_Downscaler_vs_VNA_Harvard ML_vs_Downscaler_Harvard ML': 'VNA vs eVNA aVNA Downscaler Harvard ML',
        'eVNA_aVNA_vs_eVNA_Downscaler_vs_eVNA_Harvard ML_vs_aVNA_Downscaler_vs_aVNA_Harvard ML': 'eVNA vs aVNA Downscaler Harvard ML',
        'EQUATES_VNA_vs_EQUATES_eVNA_vs_EQUATES_aVNA_vs_EQUATES_Downscaler_vs_VNA_eVNA': 'EQUATES vs VNA eVNA aVNA Downscaler vs VNA eVNA',
        'VNA_aVNA_vs_VNA_Downscaler_vs_eVNA_aVNA_vs_eVNA_Downscaler_vs_aVNA_Downscaler': 'VNA vs aVNA Downscaler eVNA aVNA eVNA Downscaler aVNA Downscaler'
    }
    
    # 根据年份确定处理模式
    if 2002 <= year <= 2016:
        # 2002-2016年的处理模式
        group_patterns = {
            'Group_1': f"Merged_{year}_{{period}}_EQUATES_VNA_vs_EQUATES_eVNA_vs_EQUATES_aVNA_vs_EQUATES_Downscaler_vs_EQUATES_Harvard ML.png",
            'Group_2': f"Merged_{year}_{{period}}_VNA_eVNA_vs_VNA_aVNA_vs_VNA_Downscaler_vs_VNA_Harvard ML_vs_Downscaler_Harvard ML.png",
            'Group_3': f"Merged_{year}_{{period}}_eVNA_aVNA_vs_eVNA_Downscaler_vs_eVNA_Harvard ML_vs_aVNA_Downscaler_vs_aVNA_Harvard ML.png"
        }
    elif 2017 <= year <= 2019:
        # 2017-2019年的处理模式
        group_patterns = {
            'Group_1': f"Merged_{year}_{{period}}_EQUATES_VNA_vs_EQUATES_eVNA_vs_EQUATES_aVNA_vs_EQUATES_Downscaler_vs_VNA_eVNA.png",
            'Group_2': f"Merged_{year}_{{period}}_VNA_aVNA_vs_VNA_Downscaler_vs_eVNA_aVNA_vs_eVNA_Downscaler_vs_aVNA_Downscaler.png"
        }
    else:
        print(f"年份 {year} 不在处理范围内")
        return
    
    # 处理每个组的图片合并
    for group_name, pattern in group_patterns.items():
        # 提取模型对比部分
        model_comparison = pattern.split('{period}_')[1].replace('.png', '')
        
        # 获取友好的模型对比名称
        friendly_name = model_comparison_names.get(model_comparison, model_comparison)
        
        for i, periods in enumerate(period_combinations, start=1):
            images = []
            for period in periods:
                image_name = pattern.format(period=period)
                image_path = os.path.join(input_base_path, group_name, image_name)
                if os.path.exists(image_path):
                    try:
                        img = Image.open(image_path)
                        images.append(img)
                    except Exception as e:
                        print(f"打开 {image_path} 时出错: {e}")
                else:
                    print(f"{image_path} 不存在。")
            
            if not images:
                print(f"对于年份 {year}、组 {group_name} 和时期 {periods}，没有可用的图片进行合并。")
                continue
            
            # 计算合并后图片的尺寸
            total_height = sum([img.height for img in images]) + (len(images) - 1) * spacing
            max_width = max([img.width for img in images])
            
            # 创建新的合并图片
            merged_image = Image.new('RGB', (max_width, total_height))
            y_offset = 0
            for img in images:
                merged_image.paste(img, (0, y_offset))
                y_offset += img.height + spacing
            
            # 保存合并后的图片，移除Group_X前缀
            output_name = f"{year}_Series{i}_{friendly_name}.png"
            # 替换文件名中的非法字符
            output_name = output_name.replace(':', '_').replace(',', '_').replace('vs.', 'vs').replace(' ', '_')
            
            output_full_path = os.path.join(output_base_path, output_name)
            try:
                merged_image.save(output_full_path)
                print(f"对于年份 {year}、组 {group_name} 和时期 {periods}，合并后的图片已保存为 {output_full_path}")
            except Exception as e:
                print(f"保存合并后的图片 {output_full_path} 时出错: {e}")

def merge_special_images(year, period_combinations, output_patterns, input_base_path, output_base_path, spacing=10):
    """
    合并特殊模式的图片
    
    参数:
    year: 处理的年份
    period_combinations: 时期组合列表
    output_patterns: 输出文件名模式
    input_base_path: 输入图片的基础路径
    output_base_path: 输出图片的基础路径
    spacing: 图片之间的间距，默认为10像素
    """
    # 特殊模式的图片处理
    special_patterns = [
        f"{year}_*_avna_ozone-vna_ozone_ds_ozone-vna_ozone_avna_ozone-evna_ozone_ds_ozone-evna_ozone_ds_ozone-avna_ozone.png",
    ]
    
    for j, special_pattern in enumerate(special_patterns, start=1):
        for i, periods in enumerate(period_combinations, start=1):
            images = []
            for period in periods:
                image_name = special_pattern.replace('*', period)
                image_path = os.path.join(input_base_path, image_name)
                if os.path.exists(image_path):
                    try:
                        img = Image.open(image_path)
                        images.append(img)
                    except Exception as e:
                        print(f"打开 {image_path} 时出错: {e}")
                else:
                    print(f"{image_path} 不存在。")
            
            if not images:
                print(f"对于年份 {year}、特殊模式 {special_pattern} 和时期 {periods}，没有可用的图片进行合并。")
                continue
            
            total_height = sum([img.height for img in images]) + (len(images) - 1) * spacing
            max_width = max([img.width for img in images])
            
            merged_image = Image.new('RGB', (max_width, total_height))
            y_offset = 0
            for img in images:
                merged_image.paste(img, (0, y_offset))
                y_offset += img.height + spacing
            
            # 从特殊模式中提取模型对比部分
            model_comparison = special_pattern.split('*_')[1].replace('.png', '')
            
            # 替换为友好的名称
            friendly_model_names = {
                'avna_ozone-vna_ozone': 'aVNA vs VNA',
                'ds_ozone-vna_ozone': 'Downscaler vs VNA',
                'avna_ozone-evna_ozone': 'aVNA vs eVNA',
                'ds_ozone-evna_ozone': 'Downscaler vs eVNA',
                'ds_ozone-avna_ozone': 'Downscaler vs aVNA'
            }
            
            # 构建友好的输出名称
            parts = model_comparison.split('_')
            friendly_parts = []
            for part in parts:
                if part in friendly_model_names:
                    friendly_parts.append(friendly_model_names[part])
                else:
                    friendly_parts.append(part)
            
            friendly_name = '_vs_'.join(friendly_parts)
            
            output_name = f"{year}_Series{i}_mergedDiff_{friendly_name}.png"
            output_full_path = os.path.join(output_base_path, output_name)
            try:
                merged_image.save(output_full_path)
                print(f"对于年份 {year}、特殊模式 {special_pattern} 和时期 {periods}，合并后的图片已保存为 {output_full_path}")
            except Exception as e:
                print(f"保存合并后的图片 {output_full_path} 时出错: {e}")

if __name__ == "__main__":
    # 设置年份范围
    years_2002_2016 = list(range(2002, 2017))  # 2002-2016
    years_2017_2019 = list(range(2017, 2020))  # 2017-2019
    
    # 基础路径设置
    input_base_path = '/DeepLearning/mnt/shixiansheng/data_fusion/output/Merged_Scatter/'
    output_base_path = '/DeepLearning/mnt/shixiansheng/data_fusion/output/MergedMerged_Scatter/'
    
    # 时期组合
    period_combinations = [
        ['DJF', 'MAM', 'JJA', 'SON'],
        ['Apr-Sep', 'Annual', 'top-10']
    ]
    
    # 输出模式（不再使用，已在函数中动态生成）
    output_patterns = []
    
    # 可以修改这个参数来调整间距
    spacing = -10
    
    # 处理2002-2016年
    for year in years_2002_2016:
        print(f"\n处理年份 {year} (2002-2016模式)")
        merge_images(year, period_combinations, output_patterns, input_base_path, output_base_path, spacing)
        merge_special_images(year, period_combinations, output_patterns, input_base_path, output_base_path, spacing)
    
    # 处理2017-2019年
    for year in years_2017_2019:
        print(f"\n处理年份 {year} (2017-2019模式)")
        merge_images(year, period_combinations, output_patterns, input_base_path, output_base_path, spacing)
        merge_special_images(year, period_combinations, output_patterns, input_base_path, output_base_path, spacing)


处理年份 2002 (2002-2016模式)
对于年份 2002、组 Group_1 和时期 ['DJF', 'MAM', 'JJA', 'SON']，合并后的图片已保存为 /DeepLearning/mnt/shixiansheng/data_fusion/output/MergedMerged_Scatter/2002_Series1_EQUATES_vs_VNA_eVNA_aVNA_Downscaler_Harvard_ML.png
对于年份 2002、组 Group_1 和时期 ['Apr-Sep', 'Annual', 'top-10']，合并后的图片已保存为 /DeepLearning/mnt/shixiansheng/data_fusion/output/MergedMerged_Scatter/2002_Series2_EQUATES_vs_VNA_eVNA_aVNA_Downscaler_Harvard_ML.png
对于年份 2002、组 Group_2 和时期 ['DJF', 'MAM', 'JJA', 'SON']，合并后的图片已保存为 /DeepLearning/mnt/shixiansheng/data_fusion/output/MergedMerged_Scatter/2002_Series1_VNA_vs_eVNA_aVNA_Downscaler_Harvard_ML.png
对于年份 2002、组 Group_2 和时期 ['Apr-Sep', 'Annual', 'top-10']，合并后的图片已保存为 /DeepLearning/mnt/shixiansheng/data_fusion/output/MergedMerged_Scatter/2002_Series2_VNA_vs_eVNA_aVNA_Downscaler_Harvard_ML.png
对于年份 2002、组 Group_3 和时期 ['DJF', 'MAM', 'JJA', 'SON']，合并后的图片已保存为 /DeepLearning/mnt/shixiansheng/data_fusion/output/MergedMerged_Scatter/2002_Series1_eVNA_vs_aVNA_Downscaler_Harvard_ML.png
对于